<a href="https://colab.research.google.com/github/kavyajeetbora/Delhi_NCR_dashboard/blob/master/notebooks/open_buildings_download_region_polygons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google open building dataset

Data format
The dataset consists of 3 parts: building polygons, building points and score thresholds.


Building polygons and points
Building polygons and points are stored in spatially sharded CSVs with one CSV per S2 cell level 4. Each row in the CSV represents one building polygon or point and has the following columns:

- **latitude**: latitude of the building polygon centroid,
- **longitude**: longitude of the building polygon centroid,
- **area_in_meters**: area in square meters of the polygon,
- **confidence**: confidence score [0.65;1.0] assigned by the model,
- **geometry**: the building polygon in the WKT format (POLYGON or MULTIPOLYGON). This feature is present in only in polygons data,
- **full_plus_code**: the full Plus Code at the building polygon centroid

### Step 1. Prepare a compressed CSV file using [Open Buildings](https://sites.research.google/open-buildings/) data [takes 1-15 minutes depending on the region]

In [ ]:
%%time
#@markdown First, select a region from either the [Natural Earth low res](https://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-admin-0-countries/) (fastest), [Natural Earth high res](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-countries/) or [World Bank high res](https://datacatalog.worldbank.org/dataset/world-bank-official-boundaries) shapefiles:
region_border_source = 'World Bank (High Res 10m)'  #@param ["Natural Earth (Low Res 110m)", "Natural Earth (High Res 10m)", "World Bank (High Res 10m)"]
region = ''  #@param ["", "AGO (Angola)", "BDI (Burundi)", "BEN (Benin)", "BFA (Burkina Faso)", "BGD (Bangladesh)", "BRN (Brunei)", "BTN (Bhutan)", "BWA (Botswana)", "CAF (Central African Republic)", "CIV (C\u00f4te d'Ivoire)", "CMR (Cameroon)", "COD (Democratic Republic of the Congo)", "COG (Republic of the Congo)", "COM (Comoros)", "CPV (Cape Verde)", "DJI (Djibouti)", "DZA (Algeria)", "EGY (Egypt)", "ERI (Eritrea)", "ETH (Ethiopia)", "GAB (Gabon)", "GHA (Ghana)", "GIN (Guinea)", "GMB (The Gambia)", "GNB (Guinea-Bissau)", "GNQ (Equatorial Guinea)", "IDN (Indonesia)", "IOT (British Indian Ocean Territory)", "KEN (Kenya)", "KHM (Cambodia)", "LAO (Laos)", "LBR (Liberia)", "LKA (Sri Lanka)", "LSO (Lesotho)", "MDG (Madagascar)", "MDV (Maldives)", "MOZ (Mozambique)", "MRT (Mauritania)", "MUS (Mauritius)", "MWI (Malawi)", "MYS (Malaysia)", "MYT (Mayotte)", "NAM (Namibia)", "NER (Niger)", "NGA (Nigeria)", "NPL (Nepal)", "PHL (Philippines)", "REU (R\u00e9union)", "RWA (Rwanda)", "SDN (Sudan)", "SEN (Senegal)", "SGP (Singapore)", "SHN (Saint Helena, Ascension and Tristan da Cunha)", "SLE (Sierra Leone)", "SOM (Somalia)", "STP (S\u00e3o Tom\u00e9 and Pr\u00edncipe)", "SWZ (Eswatini)", "SYC (Seychelles)", "TGO (Togo)", "THA (Thailand)", "TLS (Timor-Leste)", "TUN (Tunisia)", "TZA (Tanzania)", "UGA (Uganda)", "VNM (Vietnam)", "ZAF (South Africa)", "ZMB (Zambia)", "ZWE (Zimbabwe)"]
#@markdown **or** specify an area of interest in [WKT format](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) (assumes crs='EPSG:4326'); this [tool](https://arthur-e.github.io/Wicket/sandbox-gmaps3.html) might be useful.
your_own_wkt_polygon = 'POLYGON ((77.3051413673883 28.555732227913847, 77.31300984258708 28.544969186315203, 77.31438979060256 28.54374539175398, 77.32063479974838 28.538206659501633, 77.32483339884692 28.53600158105828, 77.33144856523451 28.530363873542257, 77.33649680441096 28.523759166083913, 77.3404435650323 28.518594852514614, 77.34292260123115 28.516156785945718, 77.34389780963718 28.51519772004282, 77.34415045531287 28.514752834089922, 77.3467555736686 28.510166443541074, 77.34765178578583 28.50345539386918, 77.34437051094167 28.505584654180733, 77.33889524476847 28.50282321673253, 77.33657377596872 28.49795786780577, 77.33159199450778 28.493882989277235, 77.32474908115456 28.48906551600393, 77.31672824601804 28.485191441211544, 77.31080442189256 28.484632836029512, 77.30612106397625 28.485668662545912, 77.29564126151455 28.490795216269316, 77.28790080744703 28.492677906822067, 77.28096052855314 28.491564112316944, 77.26991008923501 28.48297439297787, 77.26378569834469 28.48009357731464, 77.25244400621256 28.47915064567805, 77.24652091948481 28.474240019183075, 77.24218273391853 28.47035216349048, 77.2373837747999 28.464308874093746, 77.23549659680403 28.45856822564237, 77.2361795589214 28.454817613335457, 77.24285892495196 28.45400642953283, 77.24961574516955 28.44815818475137, 77.2530268026152 28.442036338229865, 77.25403011720421 28.436207899655177, 77.25291103475394 28.428222896042964, 77.25034156499044 28.42340186441194, 77.23243251187648 28.413519899282143, 77.22497561624277 28.410302025107384, 77.21556351810086 28.407962744783415, 77.20487044226623 28.40710099160931, 77.18913212642883 28.406957202651455, 77.1753786965703 28.403052135245247, 77.16397705230139 28.415900047311332, 77.1615020553078 28.42292256332066, 77.15205411077763 28.430329863056606, 77.14103210162064 28.433326205367408, 77.13370781022178 28.433864579383528, 77.12793400653317 28.437064448307424, 77.12215454261144 28.444875475495117, 77.1183944590692 28.451997790825555, 77.11570174637525 28.46054327066356, 77.11240519680932 28.46510414626018, 77.1112007528156 28.471134423109145, 77.1130200358822 28.477774583149763, 77.11505260605611 28.482787775065173, 77.11737741977366 28.488521191935334, 77.11321439219618 28.49212104394551, 77.10002289768582 28.500122093855918, 77.09568418742538 28.505535157149794, 77.08218639784023 28.514969729422507, 77.07717268518716 28.515336739053954, 77.06664152553824 28.50856863575967, 77.05844843242112 28.509950156564734, 77.0513371066247 28.513677486259233, 77.04283624252213 28.523067890818023, 77.02931240272851 28.528010242359837, 77.02533919539864 28.528069326699114, 77.01578176421805 28.537520967096555, 77.0098872844861 28.536763689882264, 77.00562253774439 28.531655257826593, 77.00787223541388 28.52689841227009, 77.01466123810748 28.52348360827443, 77.02105034555119 28.518854424369597, 77.02058822779065 28.512771093394687, 77.01519124948031 28.509187790973435, 76.9999072575318 28.51453985486166, 76.98617060137151 28.518579969390444, 76.97809693899958 28.522354668094167, 76.9542157538593 28.508429573644957, 76.94279847995226 28.51084937097648, 76.93223080421859 28.51408299365603, 76.91802423683191 28.509739375596542, 76.90700264854304 28.51260929400555, 76.90326649034155 28.510985967247592, 76.89633714122468 28.506079493782593, 76.888890648998 28.503906424160505, 76.88332862462205 28.509253264411544, 76.88899488687875 28.51885231626565, 76.88586106805107 28.52326006274611, 76.86877345556685 28.536804588992013, 76.86630758723604 28.542702284659295, 76.84717142496928 28.55189587437348, 76.84251435748224 28.577422597419602, 76.84464047343846 28.586319016621548, 76.8598587067249 28.585022638082673, 76.86444686291388 28.58724614495722, 76.87055846965707 28.597408516419637, 76.88765759795864 28.632525240430876, 76.90341270731547 28.62762487083816, 76.9070096334832 28.627671664479006, 76.91375069989202 28.63280689778049, 76.92009986070609 28.632690969937748, 76.93340651611648 28.62350167103948, 76.93572361453052 28.624101213998003, 76.94367249234084 28.632492501025634, 76.92798055656309 28.648132152727797, 76.92727591837325 28.652127844819315, 76.93291729164164 28.664080331174244, 76.93955322250906 28.669276359015303, 76.95208088103936 28.669941183426623, 76.95238879168892 28.669957511543057, 76.95710295606034 28.672006049929216, 76.95579604403122 28.678325016988953, 76.95824478865269 28.682834769896143, 76.95987859921074 28.68898025956203, 76.96372126982114 28.69480579748066, 76.96409474481646 28.695371953722788, 76.96450443108594 28.700214302077242, 76.9608590847245 28.70461254284807, 76.9511724371652 28.711890679258342, 76.96060604612987 28.731320662833408, 76.95915313241429 28.736356587893432, 76.96026542078114 28.741514679343066, 76.9573973479795 28.74599614812088, 76.9481689225967 28.754502444229256, 76.95658061508905 28.768908062820497, 76.95397161658781 28.773408004550145, 76.95234457925335 28.78264123627383, 76.95311925089919 28.789998472271208, 76.94750610430553 28.79838957173434, 76.94740075351014 28.804046761530323, 76.95237135087149 28.815574517806848, 76.95828556327555 28.81255028052861, 76.96584862691063 28.81490894886266, 76.96810662971122 28.825219642126683, 76.97093040117177 28.82655719312726, 76.98065730358687 28.81997686114232, 76.9837952124208 28.820710136769204, 76.98967913275564 28.831574872108764, 76.99363678918797 28.833955941949693, 76.99986666974684 28.834022508788333, 77.01676989532704 28.830110428072633, 77.0289528247558 28.831891732354034, 77.0348385350697 28.83110210017568, 77.04170609206948 28.831609861967735, 77.04786158554664 28.84759933243861, 77.05999318424534 28.86800385153041, 77.07591544442103 28.867804884159074, 77.08041327970415 28.874797294780972, 77.0861236211929 28.879321705537276, 77.0936136972754 28.86939745694931, 77.10973064204086 28.86721775214927, 77.11741355606702 28.86322476825236, 77.11997601034905 28.860891041976934, 77.13883755570389 28.86013342522956, 77.14536199101484 28.853229476866307, 77.14586011244924 28.8474284358327, 77.14406717911004 28.83944953660806, 77.14946011330588 28.836399908015032, 77.15264742713133 28.837195019299894, 77.16092954944801 28.84317053769546, 77.16990445479617 28.852147600481793, 77.17490730047335 28.85535417182215, 77.19398918870692 28.857001316197163, 77.19721033144612 28.861519456499753, 77.20383582344276 28.860039631383188, 77.20999957954142 28.85962904794304, 77.21515278829848 28.857591132498012, 77.21818736263195 28.85490179099469, 77.22180895061327 28.84960424534348, 77.22093899702955 28.83996404332024, 77.22763916539562 28.82995056625787, 77.22392665643166 28.808163422382506, 77.20488239307566 28.810023818129128, 77.20468972213968 28.796959832985785, 77.207169612488 28.789949943276, 77.21052226373159 28.78651978038676, 77.21291649218315 28.784070027014202, 77.2143440039084 28.7835189924518, 77.2186797593756 28.781845232583745, 77.2233009667238 28.781095494546847, 77.22426421506802 28.781313811917038, 77.22642010451952 28.781802345810856, 77.22851677648046 28.782277426895156, 77.2304651904059 28.782276394666184, 77.2338181975754 28.782274469435254, 77.23493032368563 28.782273792343755, 77.2323205733389 28.778380986561967, 77.2273273412673 28.776331649021603, 77.23113894140936 28.772545339912906, 77.23700717110162 28.768592574441026, 77.2392638482639 28.76707241371082, 77.2453830610995 28.75802298651334, 77.24820531120781 28.75384868265035, 77.248696429135 28.753731424176507, 77.25608762170775 28.751966179092694, 77.25868312652483 28.752152873178126, 77.26341465224546 28.74740002465139, 77.26207910610313 28.740112503092273, 77.26637280166048 28.73454010236981, 77.27012818633878 28.732723939288697, 77.27657100341791 28.7338795378577, 77.28133040053568 28.732781866462926, 77.28706457692951 28.72898544461337, 77.29089325926482 28.72461991045587, 77.29104691592772 28.724368227144385, 77.29462447352118 28.71850977850799, 77.2938530820123 28.713557060198188, 77.29509038307268 28.7070330410502, 77.30312431192496 28.70434348419933, 77.30581072337357 28.70760858582894, 77.31166707844272 28.71129384143181, 77.31643073365667 28.712231616501384, 77.32737700698904 28.711763774281142, 77.33370385341296 28.71041555057635, 77.3318683847603 28.70637169176919, 77.33003330949782 28.69819532955926, 77.33111759751431 28.694218477306148, 77.33775169207539 28.68638048190534, 77.33729417289436 28.67969463083855, 77.33578362518006 28.677391375868243, 77.3355514988024 28.677037384140963, 77.32807488393293 28.673885185094285, 77.32807361937029 28.67385857690648, 77.32800389493826 28.67239167204768, 77.32772533394477 28.666531902780093, 77.32770785324212 28.666164048955366, 77.325557835788 28.657802965445374, 77.32554397539066 28.65638446899026, 77.32553763151414 28.655733096057418, 77.32544205436459 28.645907030477645, 77.32318501604315 28.641208839049714, 77.3213046867176 28.6391865225465, 77.32367042123747 28.63774460047442, 77.323997068761 28.637426911332668, 77.3260345355317 28.635444915488762, 77.33105474972714 28.630560992626954, 77.33632519075763 28.627993545614935, 77.34055944724112 28.62593061720227, 77.34431417293108 28.62150692359716, 77.34716790087138 28.616412145357774, 77.3476231172752 28.612011393650366, 77.34625279456911 28.603392151011704, 77.34320094917466 28.601226628263596, 77.33789379673176 28.59939961184384, 77.3269718934987 28.59448299051775, 77.32337195280223 28.59397295082713, 77.31995806102422 28.593489177566717, 77.31716425390505 28.591442137128972, 77.31556413106117 28.58955191184122, 77.31515956021418 28.589074042138343, 77.30491000912598 28.576964198625987, 77.30482387610402 28.576782106473466, 77.30363037479074 28.574258900229875, 77.30375620751364 28.561264438305184, 77.3051413673883 28.555732227913847))'  #@param {type:"string"}
#@markdown Select type of data to download here:
data_type = 'polygons'  #@param ["polygons", "points"]

!sudo apt-get install swig
!pip install s2geometry pygeos geopandas

import functools
import glob
import gzip
import multiprocessing
import os
import shutil
import tempfile
from typing import List, Optional, Tuple

import geopandas as gpd
from IPython import display
import pandas as pd
import s2geometry as s2
import shapely
import tensorflow as tf
import tqdm.notebook

BUILDING_DOWNLOAD_PATH = ('gs://open-buildings-data/v3/'
                          f'{data_type}_s2_level_6_gzip_no_header')

def get_filename_and_region_dataframe(
    region_border_source: str, region: str,
    your_own_wkt_polygon: str) -> Tuple[str, gpd.geodataframe.GeoDataFrame]:
  """Returns output filename and a geopandas dataframe with one region row."""

  if your_own_wkt_polygon:
    filename = f'open_buildings_v3_{data_type}_your_own_wkt_polygon.csv.gz'
    region_df = gpd.GeoDataFrame(
        geometry=gpd.GeoSeries.from_wkt([your_own_wkt_polygon]),
        crs='EPSG:4326')
    if not isinstance(region_df.iloc[0].geometry,
                      shapely.geometry.polygon.Polygon) and not isinstance(
                          region_df.iloc[0].geometry,
                          shapely.geometry.multipolygon.MultiPolygon):
      raise ValueError("`your_own_wkt_polygon` must be a POLYGON or "
                      "MULTIPOLYGON.")
    print(f'Preparing your_own_wkt_polygon.')
    return filename, region_df

  if not region:
    raise ValueError('Please select a region or set your_own_wkt_polygon.')

  if region_border_source == 'Natural Earth (Low Res 110m)':
    url = ('https://www.naturalearthdata.com/http//www.naturalearthdata.com/'
           'download/110m/cultural/ne_110m_admin_0_countries.zip')
    !wget -N {url}
    display.clear_output()
    region_shapefile_path = os.path.basename(url)
    source_name = 'ne_110m'
  elif region_border_source == 'Natural Earth (High Res 10m)':
    url = ('https://www.naturalearthdata.com/http//www.naturalearthdata.com/'
           'download/10m/cultural/ne_10m_admin_0_countries.zip')
    !wget -N {url}
    display.clear_output()
    region_shapefile_path = os.path.basename(url)
    source_name = 'ne_10m'
  elif region_border_source == 'World Bank (High Res 10m)':
    url = ('https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/'
           '779551/wb_countries_admin0_10m.zip')
    !wget -N {url}
    !unzip -o {os.path.basename(url)}
    display.clear_output()
    region_shapefile_path = 'WB_countries_Admin0_10m'
    source_name = 'wb_10m'

  region_iso_a3 = region.split(' ')[0]
  filename = (f'open_buildings_v3_{data_type}_'
              f'{source_name}_{region_iso_a3}.csv.gz')
  region_df = gpd.read_file(region_shapefile_path).query(
      f'ISO_A3 == "{region_iso_a3}"').dissolve(by='ISO_A3')[['geometry']]
  print(f'Preparing {region} from {region_border_source}.')
  return filename, region_df


def get_bounding_box_s2_covering_tokens(
    region_geometry: shapely.geometry.base.BaseGeometry) -> List[str]:
  region_bounds = region_geometry.bounds
  s2_lat_lng_rect = s2.S2LatLngRect_FromPointPair(
      s2.S2LatLng_FromDegrees(region_bounds[1], region_bounds[0]),
      s2.S2LatLng_FromDegrees(region_bounds[3], region_bounds[2]))
  coverer = s2.S2RegionCoverer()
  # NOTE: Should be kept in-sync with s2 level in BUILDING_DOWNLOAD_PATH.
  coverer.set_fixed_level(6)
  coverer.set_max_cells(1000000)
  return [cell.ToToken() for cell in coverer.GetCovering(s2_lat_lng_rect)]


def s2_token_to_shapely_polygon(
    s2_token: str) -> shapely.geometry.polygon.Polygon:
  s2_cell = s2.S2Cell(s2.S2CellId_FromToken(s2_token, len(s2_token)))
  coords = []
  for i in range(4):
    s2_lat_lng = s2.S2LatLng(s2_cell.GetVertex(i))
    coords.append((s2_lat_lng.lng().degrees(), s2_lat_lng.lat().degrees()))
  return shapely.geometry.Polygon(coords)


def download_s2_token(
    s2_token: str, region_df: gpd.geodataframe.GeoDataFrame) -> Optional[str]:
  """Downloads the matching CSV file with polygons for the `s2_token`.

  NOTE: Only polygons inside the region are kept.
  NOTE: Passing output via a temporary file to reduce memory usage.

  Args:
    s2_token: S2 token for which to download the CSV file with building
      polygons. The S2 token should be at the same level as the files in
      BUILDING_DOWNLOAD_PATH.
    region_df: A geopandas dataframe with only one row that contains the region
      for which to keep polygons.

  Returns:
    Either filepath which contains a gzipped CSV without header for the
    `s2_token` subfiltered to only contain building polygons inside the region
    or None which means that there were no polygons inside the region for this
    `s2_token`.
  """
  s2_cell_geometry = s2_token_to_shapely_polygon(s2_token)
  region_geometry = region_df.iloc[0].geometry
  prepared_region_geometry = shapely.prepared.prep(region_geometry)
  # If the s2 cell doesn't intersect the country geometry at all then we can
  # know that all rows would be dropped so instead we can just return early.
  if not prepared_region_geometry.intersects(s2_cell_geometry):
    return None
  try:
    # Using tf.io.gfile.GFile gives better performance than passing the GCS path
    # directly to pd.read_csv.
    with tf.io.gfile.GFile(
        os.path.join(BUILDING_DOWNLOAD_PATH, f'{s2_token}_buildings.csv.gz'),
        'rb') as gf:
      # If the s2 cell is fully covered by country geometry then can skip
      # filtering as we need all rows.
      if prepared_region_geometry.covers(s2_cell_geometry):
        with tempfile.NamedTemporaryFile(mode='w+b', delete=False) as tmp_f:
          shutil.copyfileobj(gf, tmp_f)
          return tmp_f.name
      # Else take the slow path.
      # NOTE: We read in chunks to save memory.
      csv_chunks = pd.read_csv(
          gf, chunksize=2000000, dtype=object, compression='gzip', header=None)
      tmp_f = tempfile.NamedTemporaryFile(mode='w+b', delete=False)
      tmp_f.close()
      for csv_chunk in csv_chunks:
        points = gpd.GeoDataFrame(
            geometry=gpd.points_from_xy(csv_chunk[1], csv_chunk[0]),
            crs='EPSG:4326')
        # sjoin 'within' was faster than using shapely's 'within' directly.
        points = gpd.sjoin(points, region_df, predicate='within')
        csv_chunk = csv_chunk.iloc[points.index]
        csv_chunk.to_csv(
            tmp_f.name,
            mode='ab',
            index=False,
            header=False,
            compression={
                'method': 'gzip',
                'compresslevel': 1
            })
      return tmp_f.name
  except tf.errors.NotFoundError:
    return None

# Clear output after pip install.
display.clear_output()
filename, region_df = get_filename_and_region_dataframe(region_border_source,
                                                        region,
                                                        your_own_wkt_polygon)
# Remove any old outputs to not run out of disk.
for f in glob.glob('/tmp/open_buildings_*'):
  os.remove(f)
# Write header to the compressed CSV file.
with gzip.open(f'/tmp/{filename}', 'wt') as merged:
  if data_type == "polygons":
    merged.write(','.join([
        'latitude', 'longitude', 'area_in_meters', 'confidence', 'geometry',
        'full_plus_code'
    ]) + '\n')
  else:
    merged.write(','.join([
        'latitude', 'longitude', 'area_in_meters', 'confidence',
        'full_plus_code'
    ]) + '\n')
download_s2_token_fn = functools.partial(download_s2_token, region_df=region_df)
s2_tokens = get_bounding_box_s2_covering_tokens(region_df.iloc[0].geometry)
# Downloads CSV files for relevant S2 tokens and after filtering appends them
# to the compressed output CSV file. Relies on the fact that concatenating
# gzipped files produces a valid gzip file.
# NOTE: Uses a pool to speed up output preparation.
with open(f'/tmp/{filename}', 'ab') as merged:
  with multiprocessing.Pool(4) as e:
    for fname in tqdm.notebook.tqdm(
        e.imap_unordered(download_s2_token_fn, s2_tokens),
        total=len(s2_tokens)):
      if fname:
        with open(fname, 'rb') as tmp_f:
          shutil.copyfileobj(tmp_f, merged)
        os.unlink(fname)

### Step 2. Convert the data to geopandas dataframe

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

In [ ]:
%%time
df = pd.read_csv(r"/tmp/open_buildings_v3_polygons_your_own_wkt_polygon.csv.gz", compression='gzip', header=0)

In [ ]:
%%time
gdf = gpd.GeoDataFrame(df)
gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
gdf.geometry = gdf['geometry'] ## set the 'geometry' column as the default geometry
gdf = gdf.drop('full_plus_code', axis=1)
gdf.head()

In [ ]:
gdf.info()

In [ ]:
%%time
filename = "delhi_buildings.parquet"
gdf.to_parquet(filename)

def fileSize(filename):
    file_stats = os.stat(filename)
    print(f'File Size of {filename} is {file_stats.st_size / (1024 * 1024):.2f} MB')

fileSize(filename)

Writing to geopackage was taking consiferable amount of time. Hence better to use a GeoParquet file for this

In [ ]:
# %%time
# filename = "delhi_buildings.gpkg"
# gdf.to_file(filename)

# def fileSize(filename):
#     file_stats = os.stat(filename)
#     print(f'File Size of {filename} is {file_stats.st_size / (1024 * 1024):.2f} MB')

# fileSize(filename)